# Teleportation #

In [ ]:
# imports
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.misc import *;
from src.thirdparty.quantum import *;
from src.thirdparty.render import *;

from src.api.ibm import *;
from src.algorithms import *;

np.random.seed(39102901); # for repeatability

In [ ]:
# Display Teleportation Circuit
with CreateBackend(
    kind = BACKEND_SIMULATOR.AER,
) as backend:
    print('Quantumcircuit for Teleportation:');
    circuit = teleportation_protocol(include_entanglement=False);
    display(circuit.draw(output=DRAW_MODE.COLOUR.value, cregbundle=False, initial_state=False));

In [ ]:
# Run test of teleportation protocol
with CreateBackend(
    kind = BACKEND_SIMULATOR.AER,
) as backend:
    print('Quantumcircuit for testing Teleportation protocol:');
    circuit = teleportation_protocol_test();
    display(circuit.draw(output=DRAW_MODE.COLOUR.value, cregbundle=False, initial_state=True));
    # transpile circuit:
    circuit = qk_transpile(circuit, backend);
    job = backend.run(qk_assemble(circuit));
    result = job.result();
    # NOTE: qiskit orders the measured bits from buttom to top, so reverse this.
    counts = result.get_counts();
    counts = {
        key[::-1]: value for key, value in counts.items()
    };
    display(QkVisualisation.plot_histogram(counts));
    print(dedent('''
    \x1b[1mNOTE:\x1b[0m
    1) Random state \x1b[1m|ψ⟩ = U|0⟩\x1b[0m generated at start.
       To test that \x1b[1m|ψ⟩\x1b[0m was teleported,
       Bob applies the inverse of \x1b[1mU\x1b[0m and measures his state.
       If \x1b[1mcbit 3 = |0⟩\x1b[0m, then teleportation successful.
    2) We expect cbits 1+2 to be uniformly randomly distributed.
    '''))